In [1]:
from IPython.display import display, Math

latex_code = r"""
\begin{aligned}
\textbf{Decision Variables:} \quad
&c_{\text{chain}}, \; c_A, \; c_B, \; m_{\text{chain}}, \; m_A, \; m_B, \\
&u_{\text{chain}}, \; u_A, \; u_B, \; a_{\text{chain}}, \; a_A, \; a_B \;\;(\text{all integers, } \ge 0). \\[6pt]

\textbf{Volume (kg):} \quad
V_{\text{chain}} &= 1000 + 500 d_{\text{chain}} + 10 c_{\text{chain}} + 15 m_{\text{chain}} + 5 u_{\text{chain}} + 15 a_{\text{chain}}, \\
V_A &= 1000 + 2000 d_A + 5 c_A + 5 m_A + 2 u_A + 5 a_A, \\
V_B &= 1000 + 10000 d_B + 0.5 c_B + 1 m_B + 1 u_B + 3 a_B. \\[6pt]

\textbf{Revenue:} \quad
P_{\text{chain}} &= 150(1 - d_{\text{chain}}) V_{\text{chain}}, \\
P_A &= 180(1 - d_A) V_A, \\
P_B &= 200(1 - d_B) V_B, \\
P_{\text{total}} &= P_{\text{chain}} + P_A + P_B. \\[6pt]

\textbf{Cost:} \quad
C_{\text{chef}} &= 500 (c_{\text{chain}} + c_A + c_B), \\
C_{\text{macro}} &= 100000 (m_{\text{chain}} + m_A + m_B), \\
C_{\text{micro}} &= 30000 (u_{\text{chain}} + u_A + u_B), \\
C_{\text{digital}} &= 150000 a_{\text{chain}} + 50000 a_A + 5000 a_B, \\
C_{\text{total}} &= C_{\text{chef}} + C_{\text{macro}} + C_{\text{micro}} + C_{\text{digital}}. \\[6pt]

\textbf{Objective:} \quad
&\max (P_{\text{total}} - C_{\text{total}}). \\[6pt]

\textbf{Constraints:} \quad
&C_{\text{total}} \leq 500000, \\
&V_{\text{chain}} + V_A + V_B \geq 60000, \\
&c_{\text{chain}} + c_A + c_B \leq 30. \\
\end{aligned}
"""

display(Math(latex_code))



<IPython.core.display.Math object>

In [1]:
import pulp

def solve_scenario(d_chain_val, d_A_val, d_B_val):
    """
    Solve the chef/influencer/digital marketing model 
    using goal programming to handle infeasibility.
    """
    # Create an LP Problem
    prob = pulp.LpProblem("Scenario", pulp.LpMinimize)  # Now minimizing deviations
    
    # Define Decision Variables
    c_chain = pulp.LpVariable('c_chain', lowBound=0, upBound=100, cat='Integer')
    c_A = pulp.LpVariable('c_A', lowBound=0, upBound=1000, cat='Integer')
    c_B = pulp.LpVariable('c_B', lowBound=0, upBound=10000, cat='Integer')
    
    m_chain = pulp.LpVariable('m_chain', lowBound=0, upBound=100, cat='Integer')
    m_A = pulp.LpVariable('m_A', lowBound=0, upBound=1000, cat='Integer')
    m_B = pulp.LpVariable('m_B', lowBound=0, upBound=10000, cat='Integer')
    
    u_chain = pulp.LpVariable('u_chain', lowBound=0, upBound=100, cat='Integer')
    u_A = pulp.LpVariable('u_A', lowBound=0, upBound=1000, cat='Integer')
    u_B = pulp.LpVariable('u_B', lowBound=0, upBound=10000, cat='Integer')
    
    a_chain = pulp.LpVariable('a_chain', lowBound=0, upBound=100, cat='Integer')
    a_A = pulp.LpVariable('a_A', lowBound=0, upBound=1000, cat='Integer')
    a_B = pulp.LpVariable('a_B', lowBound=0, upBound=10000, cat='Integer')
    
    # Slack variables for goal programming
    volume_shortfall = pulp.LpVariable('volume_shortfall', lowBound=0, cat='Continuous')
    budget_overage = pulp.LpVariable('budget_overage', lowBound=0, cat='Continuous')
    
    # Compute Volume (kg) using fixed discounts
    volume_chain = (1000 + 500 * d_chain_val + 10*c_chain + 15*m_chain + 5*u_chain + 15*a_chain)
    volume_A = (1000 + 2000 * d_A_val + 5*c_A + 5*m_A + 2*u_A + 5*a_A)
    volume_B = (1000 + 10000 * d_B_val + 0.5*c_B + 1*m_B + 1*u_B + 3*a_B)
    
    total_volume = volume_chain + volume_A + volume_B
    
    # Price & Revenue
    revenue_chain = volume_chain * (150 * (1 - d_chain_val))
    revenue_A = volume_A * (180 * (1 - d_A_val))
    revenue_B = volume_B * (200 * (1 - d_B_val))
    total_revenue = revenue_chain + revenue_A + revenue_B
    
    # Costs
    cost_chef = 500 * (c_chain + c_A + c_B)
    cost_macro = 100000 * (m_chain + m_A + m_B)
    cost_micro = 30000 * (u_chain + u_A + u_B)
    cost_digital = 150000 * a_chain + 50000 * a_A + 5000 * a_B
    total_cost = cost_chef + cost_macro + cost_micro + cost_digital
    
    # Profit
    profit = total_revenue - total_cost
    
    # Constraints
    prob += total_volume + volume_shortfall >= 60000, "VolumeConstraint"
    prob += total_cost - budget_overage <= 500000, "BudgetConstraint"
    prob += c_chain + c_A + c_B <= 30, "ChefVisitsConstraint"
    
    # Objective: Minimize deviations
    prob += 1000 * volume_shortfall + budget_overage
    
    # Solve
    prob.solve(pulp.PULP_CBC_CMD(msg=0))
    
    # Collect results
    solution = {
        'Status': pulp.LpStatus[prob.status],
        'Profit': pulp.value(profit),
        'TotalRevenue': pulp.value(total_revenue),
        'Revenue_Chain': pulp.value(revenue_chain),
        'Revenue_A': pulp.value(revenue_A),
        'Revenue_B': pulp.value(revenue_B),
        'TotalVolume': pulp.value(total_volume),
        'VolumeShortfall': pulp.value(volume_shortfall),
        'TotalCost': pulp.value(total_cost),
        'Cost_Chef': pulp.value(cost_chef),
        'Cost_Macro': pulp.value(cost_macro),
        'Cost_Micro': pulp.value(cost_micro),
        'Cost_Digital': pulp.value(cost_digital),
        'BudgetOverage': pulp.value(budget_overage),
        'c_chain': c_chain.varValue,
        'c_A': c_A.varValue,
        'c_B': c_B.varValue,
        'm_chain': m_chain.varValue,
        'm_A': m_A.varValue,
        'm_B': m_B.varValue,
        'u_chain': u_chain.varValue,
        'u_A': u_A.varValue,
        'u_B': u_B.varValue,
        'a_chain': a_chain.varValue,
        'a_A': a_A.varValue,
        'a_B': a_B.varValue,
    }
    return solution

In [2]:
# Find best discount combination
best_profit = -float('inf')
best_solution = None
best_discounts = None

for d_chain in [0.0, 0.1, 0.2, 0.3, 0.4]:
    for d_A in [0.0, 0.1, 0.2, 0.3, 0.4]:
        for d_B in [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]:
            solution = solve_scenario(d_chain, d_A, d_B)
            if solution['Status'] == 'Optimal':
                if solution['VolumeShortfall'] == 0 and solution['BudgetOverage'] == 0:
                    if solution['Profit'] > best_profit:
                        best_profit = solution['Profit']
                        best_solution = solution
                        best_discounts = (d_chain, d_A, d_B)
                elif solution['Profit'] > best_profit:
                    best_profit = solution['Profit']
                    best_solution = solution
                    best_discounts = (d_chain, d_A, d_B)

# Display best solution found
if best_solution:
    print("BEST SCENARIO DISCOUNTS:")
    print(f"  Chain discount = {best_discounts[0]*100:.0f}%")
    print(f"  A discount     = {best_discounts[1]*100:.0f}%")
    print(f"  B discount     = {best_discounts[2]*100:.0f}%")
    print("\nRESULTS:")
    print(f"Profit: {best_solution['Profit']:.2f} TL")
    print(f"Total Revenue: {best_solution['TotalRevenue']:.2f} TL")
    print(f"Revenue - Chain: {best_solution['Revenue_Chain']:.2f} TL")
    print(f"Revenue - A: {best_solution['Revenue_A']:.2f} TL")
    print(f"Revenue - B: {best_solution['Revenue_B']:.2f} TL")
    print(f"Total Volume: {best_solution['TotalVolume']:.2f} kg")
    print(f"Volume Shortfall: {best_solution['VolumeShortfall']:.2f} kg")
    print(f"Total Cost: {best_solution['TotalCost']:.2f} TL")
    print(f"Cost - Chef: {best_solution['Cost_Chef']:.2f} TL")
    print(f"Cost - Macro: {best_solution['Cost_Macro']:.2f} TL")
    print(f"Cost - Micro: {best_solution['Cost_Micro']:.2f} TL")
    print(f"Cost - Digital: {best_solution['Cost_Digital']:.2f} TL")
    print(f"Budget Overage: {best_solution['BudgetOverage']:.2f} TL")
else:
    print("NO FEASIBLE SOLUTION FOUND")

BEST SCENARIO DISCOUNTS:
  Chain discount = 0%
  A discount     = 20%
  B discount     = 40%

RESULTS:
Profit: 531520.00 TL
Total Revenue: 1031520.00 TL
Revenue - Chain: 195000.00 TL
Revenue - A: 201600.00 TL
Revenue - B: 634920.00 TL
Total Volume: 7991.00 kg
Volume Shortfall: 52009.00 kg
Total Cost: 500000.00 TL
Cost - Chef: 15000.00 TL
Cost - Macro: 0.00 TL
Cost - Micro: 0.00 TL
Cost - Digital: 485000.00 TL
Budget Overage: 0.00 TL
